# DS-SF-36 | 04 | Databases and Scrapping | Codealong | Starter Code

## Part A | RDBMS Databases and `SQLite`

In [11]:
import os

import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

import sqlite3

> ### How to connect to a `SQLite` database

In [12]:
db = sqlite3.connect(os.path.join('..', 'datasets', 'dataset-04-zillow.db'))

> ### How to use `pandas` to run SQL queries and get the results as a `DataFrame`

In [13]:
pd.io.sql.read_sql(
'''
SELECT *
    FROM properties
    LIMIT 10
;
''', con = db)

,id,address,latitude,longitude,is_a_studio,...,size,size_unit,lot_size,lot_size_unit,built_in_year
0,2121978635,"829 Folsom St UNIT 906, San Francisco, CA",37781429,-122401860,0,...,557.0,sqft,NaN,None,2010.0
1,89239580,"690 Market St UNIT 1705, San Francisco, CA",37788246,-122403198,0,...,1050.0,sqft,NaN,None,2007.0
2,15131782,"401 Grand View Ave APT 3, San Francisco, CA",37752157,-122442356,0,...,937.0,sqft,NaN,None,1983.0
3,15179502,"250 Concord St, San Francisco, CA",37710141,-122442063,0,...,1574.0,sqft,1947.0,sqft,1959.0
4,52266124,"88 King St APT 317, San Francisco, CA",37780630,-122389635,0,...,1205.0,sqft,NaN,None,2000.0
5,2100994004,"409 Miguel St, San Francisco, CA",37736625,-122427008,0,...,400.0,sqft,NaN,None,NaN
6,15067755,"1250 Clay St APT 306, San Francisco, CA",37793575,-122413670,0,...,541.0,sqft,NaN,None,1950.0
7,15112556,"1819 26th Ave, San Francisco, CA",37753480,-122484273,0,...,1175.0,sqft,2996.0,sqft,1939.0
8,15133321,"365 Magellan Ave, San Francisco, CA",37744952,-122463110,0,...,3094.0,sqft,3920.0,sqft,1926.0
9,61288341,"710 Masonic Ave, San Francisco, CA",37774719,-122445912,0,...,1405.0,sqft,NaN,None,1914.0


### `SELECT`

In [17]:
pd.io.sql.read_sql(
'''
SELECT id, sale_price, sale_price_unit
    FROM transactions
;
''', con = db)

,id,sale_price,sale_price_unit
0,15165953,650000.00,$
1,80749447,1.15,$M
2,15155751,665000.00,$
3,15143887,2.10,$M
4,15117639,1.35,$M
...,...,...,...
995,69819708,731000.00,$
996,15076156,5.53,$M
997,119685619,625000.00,$
998,15113584,895000.00,$


### `WHERE`

In [18]:
pd.io.sql.read_sql(
'''
SELECT id, sale_price
    FROM transactions
    WHERE sale_price_unit = '$M'
;
''', con = db)

,id,sale_price
0,80749447,1.15
1,15143887,2.10
2,15117639,1.35
3,80743040,1.48
4,15134909,1.09
...,...,...
570,2101028916,2.45
571,15091950,1.75
572,89237766,1.16
573,15197693,1.36


> ### Here's let's convert the unit of `sale_price` from \$ to \$M

In [19]:
pd.io.sql.read_sql(
'''
SELECT id, sale_price / 1000000 AS sale_price
    FROM transactions
    WHERE sale_price_unit = '$'
;
''', con = db)

,id,sale_price
0,15165953,0.650
1,15155751,0.665
2,124852113,0.825
3,15124281,0.625
4,15071070,0.718
...,...,...
420,119685503,0.750
421,69819708,0.731
422,119685619,0.625
423,15113584,0.895


### `UNION`

> ### Let's combine the previous two queries into one.  `sale_price` is now in \$M accross the dataset

In [20]:
pd.io.sql.read_sql('''
SELECT id, sale_price / 1000000 AS sale_price
    FROM transactions
    WHERE sale_price_unit = '$'
UNION ALL
SELECT id, sale_price
    FROM transactions
    WHERE sale_price_unit = '$M'
;
''', con = db)

,id,sale_price
0,15165953,0.650
1,15155751,0.665
2,124852113,0.825
3,15124281,0.625
4,15071070,0.718
...,...,...
995,2101028916,2.450
996,15091950,1.750
997,89237766,1.160
998,15197693,1.360


### `WITH`

In [21]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M')

SELECT *
    FROM normalized_sale_prices
;
''', con = db)

,id,sale_price
0,15165953,0.650
1,15155751,0.665
2,124852113,0.825
3,15124281,0.625
4,15071070,0.718
...,...,...
995,2101028916,2.450
996,15091950,1.750
997,89237766,1.160
998,15197693,1.360


### `JOIN`

In [22]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M')

SELECT properties.baths AS baths, normalized_sale_prices.sale_price AS sale_price
    FROM properties
    JOIN normalized_sale_prices
        ON normalized_sale_prices.id = properties.id
;
''', con = db)

,baths,sale_price
0,1.0,0.650
1,2.0,0.665
2,NaN,0.825
3,1.0,0.625
4,1.0,0.718
...,...,...
995,3.0,2.450
996,NaN,1.750
997,2.5,1.160
998,2.0,1.360


### `WITH` (take 2)

In [23]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M'),

baths_normalized_sale_prices AS
    (SELECT properties.baths AS baths, normalized_sale_prices.sale_price AS sale_price
        FROM properties
        JOIN normalized_sale_prices
            ON normalized_sale_prices.id = properties.id)

SELECT *
    FROM baths_normalized_sale_prices
;
''', con = db)

,baths,sale_price
0,1.0,0.650
1,2.0,0.665
2,NaN,0.825
3,1.0,0.625
4,1.0,0.718
...,...,...
995,3.0,2.450
996,NaN,1.750
997,2.5,1.160
998,2.0,1.360


### `GROUP BY` and `COUNT`

> ### Let's compute the number of properties based on their number of bedrooms

In [24]:
pd.io.sql.read_sql(
'''
SELECT COUNT() AS count
    FROM properties
    GROUP BY beds
;
''', con = db)

,count
0,164
1,142
2,316
3,226
4,103
5,28
6,9
7,6
8,3
9,3


### `GROUP BY` and `AVG`

> ### Let's now compute the average sale price of these properties based on their number of bathrooms

In [25]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT id, sale_price / 1000000 AS sale_price
        FROM transactions
        WHERE sale_price_unit = '$'
    UNION ALL
    SELECT id, sale_price
        FROM transactions
        WHERE sale_price_unit = '$M'),

baths_normalized_sale_prices AS
    (SELECT properties.baths AS baths, normalized_sale_prices.sale_price AS sale_price
        FROM properties
        JOIN normalized_sale_prices
            ON normalized_sale_prices.id = properties.id)

SELECT AVG(sale_price) AS mean_sale_price
    FROM baths_normalized_sale_prices
    GROUP BY baths
;
''', con = db)

,mean_sale_price
0,1.739026
1,0.987656
2,1.420000
3,1.600000
4,1.223378
...,...
16,16.000000
17,0.999000
18,5.530000
19,13.100000


Note: `baths = 0` usually refers to studios but not always...

> ### Activity
> #### Compute the average sales price of studios

In [35]:
pd.io.sql.read_sql(
'''
SELECT 
sum(case when sale_price_unit = '$' then sale_price else sale_price*1000000 end)
/count(distinct id) as AvgPrice   FROM transactions where 1=1 
;
''', con = db)

,AvgPrice
0,1397422.943


> ### Activity
> #### Find the properties in the dataset that have different addresses but the same latitude/longitude.  Return the smallest set of pair of properties (`address_a`, `address_b`)

In [43]:
# TODO
pd.io.sql.read_sql(
'''
SELECT
    P1.address, P2.address
   FROM
       properties p1  left join properties p2  on P1.id = P2.id
   WHERE 1=1
   and upper(p1.address) <> upper(p2.address)
   and p1.latitude = p2.latitude
   and p1.longitude = p2.longitude
   GROUP BY P1.address, P2.address
   ORDER BY P1.address, P2.address
;
''', con = db)

,address,address


In [36]:
# TODO
pd.io.sql.read_sql(
'''
select * from properties 
;
''', con = db)

,id,address,latitude,longitude,is_a_studio,...,size,size_unit,lot_size,lot_size_unit,built_in_year
0,2121978635,"829 Folsom St UNIT 906, San Francisco, CA",37781429,-122401860,0.0,...,557.0,sqft,NaN,None,2010.0
1,89239580,"690 Market St UNIT 1705, San Francisco, CA",37788246,-122403198,0.0,...,1050.0,sqft,NaN,None,2007.0
2,15131782,"401 Grand View Ave APT 3, San Francisco, CA",37752157,-122442356,0.0,...,937.0,sqft,NaN,None,1983.0
3,15179502,"250 Concord St, San Francisco, CA",37710141,-122442063,0.0,...,1574.0,sqft,1947.0,sqft,1959.0
4,52266124,"88 King St APT 317, San Francisco, CA",37780630,-122389635,0.0,...,1205.0,sqft,NaN,None,2000.0
...,...,...,...,...,...,...,...,...,...,...,...
995,82786211,"310 Townsend St APT 311, San Francisco, CA",37777027,-122395736,0.0,...,853.0,sqft,NaN,None,2006.0
996,15103435,"1343 31st Ave, San Francisco, CA",37762152,-122490254,0.0,...,1886.0,sqft,3000.0,sqft,1934.0
997,15195183,"3916 Alemany Blvd, San Francisco, CA",37711527,-122467755,0.0,...,1300.0,sqft,2553.0,sqft,1941.0
998,15180783,"430 Fair Oaks St, San Francisco, CA",37749725,-122424094,0.0,...,2678.0,sqft,NaN,None,1911.0


In [10]:
#Trying to run ebay code
#Library import
import pyodbc
pyodbc.pooling = False
import pyodbc, os, string, sys, datetime, calendar, time, shutil, traceback
from ConfigParser import SafeConfigParser
from datetime import timedelta
from datetime import date
from time import sleep

#Environment variable
def get_conn_string(env_str):
    username = "issingh"
    password = "JaiAnita996$"
    if env_str == "mz":
        return "DSN=mozart;Database=ACCESS_VIEWS;UID=" + username + ";PWD=" + password + ";"
    if env_str == "hp":
        return "DSN=hopper;Database=ACCESS_VIEWS;UID=" + username + ";PWD=" + password + ";"

#create methods to connect and disconnect from Teradata
def create_connection (box):
    conn_string = get_conn_string(box)
    cnxn = pyodbc.connect(conn_string)
    return cnxn.cursor(), cnxn

def kill_connection (cursor, cnxn):
    cursor.close()
    cnxn.close()


cursor, cnxn = create_connection("mz")  # change connection string as required

str_sql_1 = """
SELECT * FROM app_harmony_t.CMSCR_ONLINE_TRFC_BY_ENTITY 
    WHERE MONTH_BEG_DT = '2016-06-01'
    AND CNTRY_ID = 1
    AND ENTITY_NAME  LIKE '%stubhub%'
    """
cursor.execute(str_sql_1)
cnxn.commit()

kill_connection(cursor, cnxn)


Error: ('IM002', '[IM002] [unixODBC][Driver Manager]Data source name not found, and no default driver specified (0) (SQLDriverConnect)')